<a href="https://cognitiveclass.ai"><img src = "https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/Logos/organization_logo/organization_logo.png" width = 400> </a>

<h1 align=center><font size = 5>Peer Review Final Assignment</font></h1>

## Introduction


In this lab, you will build an image classifier using the VGG16 pre-trained model, and you will evaluate it and compare its performance to the model we built in the last module using the ResNet50 pre-trained model. Good luck!

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>    

1. <a href="#item41">Download Data 
2. <a href="#item42">Part 1</a>
3. <a href="#item43">Part 2</a>  
4. <a href="#item44">Part 3</a>  

</font>
    
</div>

<a id="item41"></a>

## Download Data

Use the <code>wget</code> command to download the data for this assignment from here: https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip

Use the following cells to download the data.

In [1]:
# get the data
!wget https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip

--2020-09-04 00:46:17--  https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip
Resolving s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)... 67.228.254.196
Connecting to s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)|67.228.254.196|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 261483817 (249M) [application/zip]
Saving to: ‘concrete_data_week4.zip.1’

100%[======================================>] 261,483,817 42.3MB/s   in 6.0s   

2020-09-04 00:46:23 (41.7 MB/s) - ‘concrete_data_week4.zip.1’ saved [261483817/261483817]



In [2]:
!unzip concrete_data_week4.zip

Archive:  concrete_data_week4.zip
replace concrete_data_week4/valid/positive/16679_1.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


After you unzip the data, you fill find the data has already been divided into a train, validation, and test sets.

<a id="item42"></a>

## Part 1

In this part, you will design a classifier using the VGG16 pre-trained model. Just like the ResNet50 model, you can import the model <code>VGG16</code> from <code>keras.applications</code>.

You will essentially build your classifier as follows:
1. Import libraries, modules, and packages you will need. Make sure to import the *preprocess_input* function from <code>keras.applications.vgg16</code>.
2. Use a batch size of 100 images for both training and validation.
3. Construct an ImageDataGenerator for the training set and another one for the validation set. VGG16 was originally trained on 224 × 224 images, so make sure to address that when defining the ImageDataGenerator instances.
4. Create a sequential model using Keras. Add VGG16 model to it and dense layer.
5. Compile the mode using the adam optimizer and the categorical_crossentropy loss function.
6. Fit the model on the augmented data using the ImageDataGenerators.

Use the following cells to create your classifier.

In [1]:
from keras.preprocessing.image import ImageDataGenerator
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.applications import ResNet50, VGG16
from keras.applications.resnet50 import preprocess_input
from keras.applications.vgg16 import preprocess_input

Using TensorFlow backend.


In [2]:
num_classes = 2

image_resize = 224

batch_size_training = 100
batch_size_validation = 100

In [10]:
!ls ./concrete_data_week4

test  train  valid


In [3]:
data_generator = ImageDataGenerator(
    preprocessing_function=preprocess_input,
)

train_generator = data_generator.flow_from_directory(
    'concrete_data_week4/train',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_training,
    class_mode='categorical')


validation_generator = data_generator.flow_from_directory(
    'concrete_data_week4/valid',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_validation,
    class_mode='categorical')


Found 30001 images belonging to 2 classes.
Found 9501 images belonging to 2 classes.


In [4]:
model_r = Sequential()

model_r.add(ResNet50(
    include_top=False,
    pooling='avg',
    weights='imagenet',
    ))
model_r.add(Dense(num_classes, activation='softmax'))

model_v = Sequential()
model_v.add(VGG16(
    include_top=False,
    pooling='avg',
    weights='imagenet',
    ))
model_v.add(Dense(num_classes, activation='softmax'))

Instructions for updating:
Colocations handled automatically by placer.


In [5]:
model_r.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model_v.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [6]:
steps_per_epoch_training = len(train_generator)
steps_per_epoch_validation = len(validation_generator)
num_epochs = 2

In [ ]:
fit_history_r = model_r.fit_generator(
    train_generator,
    steps_per_epoch=steps_per_epoch_training,
    epochs=num_epochs,
    validation_data=validation_generator,
    validation_steps=steps_per_epoch_validation,
    verbose=1,
)

Instructions for updating:
Use tf.cast instead.
Epoch 1/2


In [ ]:
model_r.save('classifier_resnet_model.h5')

In [ ]:
fit_history_v = model_v.fit_generator(
    train_generator,
    steps_per_epoch=steps_per_epoch_training,
    epochs=num_epochs,
    validation_data=validation_generator,
    validation_steps=steps_per_epoch_validation,
    verbose=1,
)

In [ ]:
model_v.save('classifier_vgg16_model.h5')

<a id="item43"></a>

## Part 2

In this part, you will evaluate your deep learning models on a test data. For this part, you will need to do the following:

1. Load your saved model that was built using the ResNet50 model. 
2. Construct an ImageDataGenerator for the test set. For this ImageDataGenerator instance, you only need to pass the directory of the test images, target size, and the **shuffle** parameter and set it to False.
3. Use the **evaluate_generator** method to evaluate your models on the test data, by passing the above ImageDataGenerator as an argument. You can learn more about **evaluate_generator** [here](https://keras.io/models/sequential/).
4. Print the performance of the classifier using the VGG16 pre-trained model.
5. Print the performance of the classifier using the ResNet pre-trained model.


Use the following cells to evaluate your models.

In [ ]:
model_r.load('classifier_resnet_model.h5')
model_v.load('classifier_vgg16_model.h5')

In [7]:
data_generator = ImageDataGenerator(
    preprocessing_function=preprocess_input,
)

test_generator = data_generator.flow_from_directory(
    'concrete_data_week4/test',
    target_size=(image_resize, image_resize),
    shuffle = False)

Found 500 images belonging to 2 classes.


In [12]:
steps_per_epoch_test = len(test_generator)

In [15]:
eval_history_r = model_r.evaluate_generator(
    test_generator,
    steps=steps_per_epoch_test,
    verbose=1,
)


eval_history_v = model_v.evaluate_generator(
    test_generator,
    steps=steps_per_epoch_test,
    verbose=1,
)

16/16 [==============================] - 159s 10s/step


In [17]:
print(eval_history_r)
print(eval_history_v)

[1.0048215532302855, 0.36200000005960464]
[0.7750662353038787, 0.5400000009536743]


<a id="item44"></a>

## Part 3

In this model, you will predict whether the images in the test data are images of cracked concrete or not. You will do the following:

1. Use the **predict_generator** method to predict the class of the images in the test data, by passing the test data ImageDataGenerator instance defined in the previous part as an argument. You can learn more about the **predict_generator** method [here](https://keras.io/models/sequential/).
2. Report the class predictions of the first five images in the test set. You should print something list this:

<center>
    <ul style="list-style-type:none">
        <li>Positive</li>  
        <li>Negative</li> 
        <li>Positive</li>
        <li>Positive</li>
        <li>Negative</li>
    </ul>
</center>

Use the following cells to make your predictions.

In [20]:
steps_per_epoch_test = len(test_generator)

pred_history_r = model_r.predict_generator(
    test_generator,
    steps = steps_per_epoch_test,
    verbose=1
)

pred_history_v = model_v.predict_generator(
    test_generator,
    steps = steps_per_epoch_test,
    verbose=1
)

16/16 [==============================] - 159s 10s/step


In [21]:
print(pred_history_r)

[[0.4892766  0.51072335]
 [0.390175   0.60982496]
 [0.576937   0.42306295]
 [0.7047237  0.2952763 ]
 [0.6519897  0.3480103 ]
 [0.35587323 0.6441268 ]
 [0.56379765 0.43620235]
 [0.46964583 0.5303542 ]
 [0.38772777 0.61227226]
 [0.55548126 0.44451877]
 [0.39983755 0.60016245]
 [0.5575021  0.4424979 ]
 [0.4841633  0.5158367 ]
 [0.65623474 0.34376526]
 [0.754087   0.24591304]
 [0.49315503 0.50684494]
 [0.8011702  0.19882986]
 [0.16871473 0.8312853 ]
 [0.268515   0.73148495]
 [0.6185272  0.38147283]
 [0.3134176  0.6865824 ]
 [0.37958735 0.62041265]
 [0.64006126 0.35993877]
 [0.90315056 0.09684943]
 [0.6261956  0.37380436]
 [0.46133372 0.5386662 ]
 [0.53067297 0.469327  ]
 [0.7079203  0.29207966]
 [0.45342067 0.54657936]
 [0.21999483 0.7800052 ]
 [0.48071176 0.51928824]
 [0.60762393 0.39237604]
 [0.5374442  0.4625559 ]
 [0.87876564 0.12123434]
 [0.56663644 0.43336362]
 [0.48672304 0.513277  ]
 [0.3464905  0.6535095 ]
 [0.7356613  0.26433867]
 [0.698017   0.30198297]
 [0.7526414  0.24735868]


In [22]:
print(pred_history_v)

[[3.42837572e-01 6.57162488e-01]
 [2.69114733e-01 7.30885208e-01]
 [3.84832412e-01 6.15167558e-01]
 [1.18768573e-01 8.81231487e-01]
 [2.46581435e-01 7.53418624e-01]
 [5.22606671e-01 4.77393359e-01]
 [3.31979811e-01 6.68020189e-01]
 [4.18167919e-01 5.81832170e-01]
 [2.85024285e-01 7.14975655e-01]
 [1.06937505e-01 8.93062472e-01]
 [1.55344859e-01 8.44655156e-01]
 [4.13477123e-02 9.58652258e-01]
 [1.52869761e-01 8.47130239e-01]
 [5.36340833e-01 4.63659137e-01]
 [4.23235476e-01 5.76764464e-01]
 [6.94925129e-01 3.05074900e-01]
 [1.06614947e-01 8.93385053e-01]
 [6.51841283e-01 3.48158717e-01]
 [3.04280490e-01 6.95719481e-01]
 [3.92243713e-01 6.07756257e-01]
 [3.31931889e-01 6.68068111e-01]
 [5.15124500e-01 4.84875530e-01]
 [3.69438678e-01 6.30561292e-01]
 [3.09289813e-01 6.90710187e-01]
 [4.99905050e-01 5.00094891e-01]
 [6.13426447e-01 3.86573583e-01]
 [5.33553362e-01 4.66446579e-01]
 [2.73972243e-01 7.26027727e-01]
 [1.33234680e-01 8.66765380e-01]
 [4.52572048e-01 5.47427893e-01]
 [2.104460

### Thank you for completing this lab!

This notebook was created by Alex Aklson.

This notebook is part of a course on **Coursera** called *AI Capstone Project with Deep Learning*. If you accessed this notebook outside the course, you can take this course online by clicking [here](https://cocl.us/DL0321EN_Coursera_Week4_LAB1).

<hr>

Copyright &copy; 2020 [IBM Developer Skills Network](https://cognitiveclass.ai/?utm_source=bducopyrightlink&utm_medium=dswb&utm_campaign=bdu). This notebook and its source code are released under the terms of the [MIT License](https://bigdatauniversity.com/mit-license/).